# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [36]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'D:/GitHub/Data/Data1/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [37]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
 
mean_df = df.groupby(['Pclass'])['Age'].mean().reset_index()
mode_df = df.groupby(['Pclass'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Pclass'])['Age'].median().reset_index()
max_df = df.groupby(['Pclass'])['Age'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Pclass'])
temp = pd.merge(temp, median_df, how='left', on=['Pclass'])
temp = pd.merge(temp, max_df, how='left', on=['Pclass'])
temp.columns = ['Pclass', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max']
temp

,Pclass,Age_Mean,Age_Mode,Age_Median,Age_Max
0,1,38.233441,35.0,37.0,80.0
1,2,29.877630,24.0,29.0,70.0
2,3,25.140620,22.0,24.0,74.0


In [41]:
df = pd.merge(df, temp, how='left', on=['Pclass'])
df = df.drop(['Pclass'] , axis=1)
 

KeyError: 'Pclass'

In [42]:
df

,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Mean,Age_Mode,Age_Median,Age_Max
0,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,25.140620,22.0,24.0,74.0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,38.233441,35.0,37.0,80.0
2,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,25.140620,22.0,24.0,74.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,38.233441,35.0,37.0,80.0
4,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,25.140620,22.0,24.0,74.0
5,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,25.140620,22.0,24.0,74.0
6,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,38.233441,35.0,37.0,80.0
7,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,25.140620,22.0,24.0,74.0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,25.140620,22.0,24.0,74.0
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,29.877630,24.0,29.0,70.0


In [43]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

8 Numeric Features : ['Age', 'SibSp', 'Parch', 'Fare', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max']



,Age,SibSp,Parch,Fare,Age_Mean,Age_Mode,Age_Median,Age_Max
0,22.0,1,0,7.2500,25.140620,22.0,24.0,74.0
1,38.0,1,0,71.2833,38.233441,35.0,37.0,80.0
2,26.0,0,0,7.9250,25.140620,22.0,24.0,74.0
3,35.0,1,0,53.1000,38.233441,35.0,37.0,80.0
4,35.0,0,0,8.0500,25.140620,22.0,24.0,74.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [45]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

df_minus = df.drop(['Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max'] , axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.055727824174331086

In [46]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

# 新特徵 + 線性迴歸 : 有些為改善
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.11809602422182694